<a href="https://colab.research.google.com/github/ThomasAlbin/Astroniz-YT-Tutorials/blob/main/CompressedCosmos/CompressedCosmos_The_Blaze_Star.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T Coronae Borealis

T Coronae Borealis (T CrB), also known as *Blaze Star*, is a star that is mostly rather faint and not visible to the naked eye. However, currently (mid 2024) a potential Nova is emerging that will lead to an illumination increase. The Blaze Star will become visivle to the naked eye!

But... where is the star? Well that's the purpose of this Python script!

In [ ]:
!pip install skyfield
!pip install astroquery
!pip install astropy

In [ ]:
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord

import skyfield
import skyfield.api
from skyfield.data import hipparcos
from skyfield.data import stellarium
from skyfield.projections import build_stereographic_projection
from skyfield.data import hipparcos as skyfield_hipparcos

from matplotlib.collections import LineCollection
import matplotlib.pyplot as plt
import numpy as np

import re

In [ ]:
# Define target star and constellation
target_star = SkyCoord.from_name("T Coronae Borealis")
constellation = "CrB"  # Abbreviation for Corona Borealis

In [ ]:
# Get star data
with skyfield.api.load.open(skyfield_hipparcos.URL) as _obj:
    stars_df = skyfield_hipparcos.load_dataframe(_obj)

stars_df

In [ ]:
# Set up Simbad query
customSimbad = Simbad()
customSimbad.add_votable_fields('ids')  # Request identifier information

# Query for T Coronae Borealis
result_table = customSimbad.query_object("T Coronae Borealis")

# Search for tcrb
hipparcos_id = int(re.search(r"HIP (\d+)", result_table["IDS"][0]).group(1))
print(f"The Hipparcos ID of T Coronae Borealis is: {hipparcos_id}")
tcrb = stars_df[stars_df.index==hipparcos_id].copy()

In [ ]:
result_table["IDS"]

In [ ]:
# Load Skyfield data
load = skyfield.api.Loader('~/skyfield_data')
ts = load.timescale()
eph = load('de421.bsp')
earth = eph['earth']

# Load data
ts = load.timescale()
t = ts.now()  # Current time

with load.open(hipparcos.URL) as f:
    stars = hipparcos.load_dataframe(f)

# Constellation outlines come from Stellarium
url = ('https://raw.githubusercontent.com/Stellarium/stellarium/master'
       '/skycultures/modern_st/constellationship.fab')

with load.open(url) as f:
    constellations = stellarium.parse_constellations(f)

edges = [edge for name, edges in constellations for edge in edges]
edges_star1 = [star1 for star1, star2 in edges]
edges_star2 = [star2 for star1, star2 in edges]

# Build projection centered at 0h RA, 0° DEC
tcrb_star = skyfield.api.Star.from_dataframe(tcrb)
tcrb_apparent = earth.at(t).observe(tcrb_star)
projection = build_stereographic_projection(tcrb_apparent)  # Use the apparent position
field_of_view_degrees = 45.0
limiting_magnitude = 6.5

# Compute star positions
star_positions = earth.at(t).observe(skyfield.api.Star.from_dataframe(stars))
stars['x'], stars['y'] = projection(star_positions)

bright_stars = (stars.magnitude <= limiting_magnitude)
magnitude = stars['magnitude'][bright_stars]
marker_size = (0.5 + limiting_magnitude - magnitude) ** 2.0

# The constellation lines will each begin at the x,y of one star and end
# at the x,y of another.  We have to "rollaxis" the resulting coordinate
# array into the shape that matplotlib expects.
xy1 = stars[['x', 'y']].loc[edges_star1].values
xy2 = stars[['x', 'y']].loc[edges_star2].values
lines_xy = np.rollaxis(np.array([xy1, xy2]), 1)

# Time to build the figure!
plt.style.use("dark_background")
fig, ax = plt.subplots(figsize=[9, 9])

# Draw the constellation lines.
ax.add_collection(LineCollection(lines_xy, colors="tab:red", alpha=.8))

# Draw the stars.
ax.scatter(stars['x'][bright_stars], stars['y'][bright_stars],
           s=marker_size, color="tab:red")

# Add the Star, which is the center of the plot
ax.scatter(0.0, 0.0, marker='+', s=50, c="white")

# Finally, title the plot and set some final parameters.
angle = np.pi - field_of_view_degrees / 360.0 * np.pi
limit = np.sin(angle) / (1.0 - np.cos(angle))

ax.set_xlim(-limit, limit)
ax.set_ylim(-limit, limit)
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
ax.set_aspect(1.0)

# Remove Frame
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)